In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist, cosine
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import SpectralClustering
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import tensorflow as tf
from joblib import Parallel, delayed

from matplotlib import cm
import matplotlib.cbook as cbook
import matplotlib.colors as colors

import pandas as pd
import xarray as xr
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime, timedelta
import os

import seaborn as sns
import cmocean as cmocn

%matplotlib inline

2024-08-08 13:21:49.988304: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-08 13:21:50.018197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 13:21:50.018226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 13:21:50.019621: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-08 13:21:50.024864: I tensorflow/core/platform/cpu_feature_gua

In [2]:
ds = xr.open_dataset('../rainFlagRemovedBuoyDataBadQualityRemovedMatchup.nc')
df = ds.to_dataframe()
df_orig = df.copy()

In [3]:
df['Speed Difference (QuikSCAT - TAO)'] = df['Wind Speed (QuikSCAT)'] - df['Wind Speed (TAO)']

df['Wind Direction (TAO)'] = (-(df['Wind Direction (TAO)'] - 90.0) + 360)%360
df['Wind Direction (QuikSCAT)'] = (-(df['Wind Direction (QuikSCAT)'] - 90.0) + 360)%360
df['mean WDIR 30min'] = (-(df['mean WDIR 30min'] - 90.0) + 360)%360
df['mean WDIR 60min'] = (-(df['mean WDIR 60min'] - 90.0) + 360)%360
df['mean WDIR 120min'] = (-(df['mean WDIR 120min'] - 90.0) + 360)%360

df['Direction Difference (QuikSCAT - TAO)'] = ((df['Wind Direction (QuikSCAT)'] - df['Wind Direction (TAO)'])+360)%360
dirDiff = np.array(df['Direction Difference (QuikSCAT - TAO)'])
dirDiff[dirDiff > 180] -= 360
df['Direction Difference (QuikSCAT - TAO)'] = dirDiff

df['Speed Difference (QuikSCAT - TAO 30 min mean)'] = df['Wind Speed (QuikSCAT)'] - df['mean WSPD 30min']
df['Direction Difference (QuikSCAT - TAO 30 min mean)'] = ((df['Wind Direction (QuikSCAT)'] - df['mean WDIR 30min'])+360)%360
dirDiff = np.array(df['Direction Difference (QuikSCAT - TAO 30 min mean)'])
dirDiff[dirDiff > 180] -= 360
df['Direction Difference (QuikSCAT - TAO 30 min mean)'] = dirDiff

df['Speed Difference (QuikSCAT - TAO 1 hr mean)'] = df['Wind Speed (QuikSCAT)'] - df['mean WSPD 60min']
df['Direction Difference (QuikSCAT - TAO 1 hr mean)'] = ((df['Wind Direction (QuikSCAT)'] - df['mean WDIR 60min'])+360)%360
dirDiff = np.array(df['Direction Difference (QuikSCAT - TAO 1 hr mean)'])
dirDiff[dirDiff > 180] -= 360
df['Direction Difference (QuikSCAT - TAO 1 hr mean)'] = dirDiff

df['Speed Difference (QuikSCAT - TAO 2 hr mean)'] = df['Wind Speed (QuikSCAT)'] - df['mean WSPD 120min']
df['Direction Difference (QuikSCAT - TAO 2 hr mean)'] = ((df['Wind Direction (QuikSCAT)'] - df['mean WDIR 120min'])+360)%360
dirDiff = np.array(df['Direction Difference (QuikSCAT - TAO 2 hr mean)'])
dirDiff[dirDiff > 180] -= 360
df['Direction Difference (QuikSCAT - TAO 2 hr mean)'] = dirDiff


df['Zonal Neutral Wind Speed at 10m (TAO)'] = df['Neutral Wind Speed at 10m (TAO)']*np.cos(np.deg2rad(df['Wind Direction (TAO)']))
df['Meridional Neutral Wind Speed at 10m (TAO)'] = df['Neutral Wind Speed at 10m (TAO)']*np.sin(np.deg2rad(df['Wind Direction (TAO)']))

df['Zonal Neutral Wind Speed at 10m (QuikSCAT)'] = df['Wind Speed (QuikSCAT)']*np.cos(np.deg2rad(df['Wind Direction (QuikSCAT)']))
df['Meridional Neutral Wind Speed at 10m (QuikSCAT)'] = df['Wind Speed (QuikSCAT)']*np.sin(np.deg2rad(df['Wind Direction (QuikSCAT)']))

df['Zonal Wind Speed Difference (QuikSCAT - TAO)'] = df['Zonal Neutral Wind Speed at 10m (QuikSCAT)'] - df['Zonal Neutral Wind Speed at 10m (TAO)']
df['Meridional Wind Speed Difference (QuikSCAT - TAO)'] = df['Meridional Neutral Wind Speed at 10m (QuikSCAT)'] - df['Meridional Neutral Wind Speed at 10m (TAO)']

df['cos(Direction Difference (QuikSCAT - TAO))'] = np.cos(np.deg2rad(df['Direction Difference (QuikSCAT - TAO)']))
df['sin(Direction Difference (QuikSCAT - TAO))'] = np.sin(np.deg2rad(df['Direction Difference (QuikSCAT - TAO)']))

In [4]:
def plotSpeedDirectionCluster(lat, lon, df):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    if isPresent:   
        selectX = [#'Speed Difference (QuikSCAT - TAO)']#,
                   'cos(Direction Difference (QuikSCAT - TAO))',
                   'sin(Direction Difference (QuikSCAT - TAO))']

        X = subDF[selectX]
        
        normX = (X - X.mean(axis=0))/ X.std(axis=0)
        
        # kmeans = KMeans(n_clusters=9, random_state=0, n_init="auto", max_iter = 10000).fit(normX)
        # subDF['label'] = kmeans.labels_

        dbscan = DBSCAN(eps=0.05, min_samples= 10)
        subDF['label'] = dbscan.fit_predict(normX)
        
        fig,  axes = plt.subplots(ncols = 2, figsize=(16,6))
        
        ax = axes[0]
        xlabel = 'Speed Difference (QuikSCAT - TAO)'
        ylabel = 'Direction Difference (QuikSCAT - TAO)'
        
        s = sns.histplot(subDF, x=xlabel, y = ylabel,#levels=10, 
                        hue='label', common_norm=False,
                        palette = sns.color_palette("bright"), ax = ax)
        
        ax.grid(visible=True, which='both', axis='both')
        
        ax.text(0.1, 0.95, f'A {lon:4d}, {lat:4d}', horizontalalignment='left',
                verticalalignment='center', transform=ax.transAxes,
                weight = 'heavy', fontsize = 20)
        
        #ax.set_xlim(-10,15)
        #ax1.set_title(f'{lon:4d}, {lat:4d}')
        
        
        ax = axes[1]
        s = sns.countplot(subDF, x='label',palette = sns.color_palette("bright"), ax = ax)
        
        ax.grid(visible=True, which='both', axis='both')
        ax.set_xlabel('cluster label')
        #ax.set_ylabel('cluster label')
        
        ax.text(0.1, 0.95, 'B', horizontalalignment='left',
                verticalalignment='center', transform=ax.transAxes,
                weight = 'heavy', fontsize = 20)
        
        plt.savefig(f'ClusterDirectionOnly_Fig{lon}lon_{lat}lat.png')
        plt.close()

    

In [ ]:
# latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
# lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

# ylen = len(latList)
# xlen = len(lonList)

# taskList = []

# for latId  in range(ylen):
#     for lonId in range(xlen):
#         taskList.append([latList[latId], lonList[lonId]])
# for latId  in range(ylen):
#     for lonId in range(xlen):
#         taskList.append([latList[latId], lonList[lonId]])

# for task in taskList:
#     lat = task[0]
#     lon = task[1]
#     plotSpeedDirectionCluster(lat, lon, df)
    

In [ ]:
df.keys()

In [ ]:
def getLabelledDB(lat, lon, df):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    
    if len(subDF) > 0:
        isPresent = True
    if isPresent:   
        selectX = [#'Speed Difference (QuikSCAT - TAO)']#,
                   'cos(Direction Difference (QuikSCAT - TAO))',
                   'sin(Direction Difference (QuikSCAT - TAO))']

        X = subDF[selectX]
        
        normX = (X - X.mean(axis=0))/ X.std(axis=0)

        dbscan = DBSCAN(eps=0.25, min_samples= 10)
        subDF['db_speed_label'] = dbscan.fit_predict(normX)


        selectX = ['Speed Difference (QuikSCAT - TAO)']#,
                   # 'cos(Direction Difference (QuikSCAT - TAO))',
                   # 'sin(Direction Difference (QuikSCAT - TAO))']

        X = subDF[selectX]
        
        normX = (X - X.mean(axis=0))/ X.std(axis=0)

        dbscan = DBSCAN(eps=0.25, min_samples= 10)
        subDF['db_direction_label'] = dbscan.fit_predict(normX)

        selectX = ['Speed Difference (QuikSCAT - TAO)',
                   'cos(Direction Difference (QuikSCAT - TAO))',
                   'sin(Direction Difference (QuikSCAT - TAO))']

        X = subDF[selectX]
        
        normX = (X - X.mean(axis=0))/ X.std(axis=0)

        dbscan = DBSCAN(eps=0.15, min_samples= 10)
        subDF['db_speedDirection_label'] = dbscan.fit_predict(normX)
        
        return subDF, len(subDF)
    else:
        return -999, -999
        
        

In [ ]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

labelledDF = []
count = 0
for task in taskList:
    lat = task[0]
    lon = task[1]
    rdf, c  = getLabelledDB(lat, lon, df)
    if isinstance(rdf, pd.DataFrame): 
        labelledDF.append(rdf)
        count += c
        print(count)
        
lbDF = pd.concat(labelledDF )
    

In [ ]:
lbDF

In [ ]:
def plotSpeedDirection(lat, lon, df):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    if isPresent:   
        fig, axes = plt.subplots(nrows=4, ncols=1, figsize = (20, 14))
        ax1 = axes[0]
        ax2 = axes[1]
        ax3 = axes[2]
        ax4 = axes[3]
        
        ax1.plot(subDF['QS_TIME'], subDF['Wind Speed (QuikSCAT)'], label = 'QuikSCAT', alpha = 0.5)
        ax1.plot(subDF['QS_TIME'], subDF['Wind Speed (TAO)'], label = 'TAO', alpha = 0.5)
        ax1.legend()
        
        ax2.plot(subDF['QS_TIME'], subDF['Wind Direction (QuikSCAT)'], label = 'QuikSCAT', alpha = 0.5)
        ax2.plot(subDF['QS_TIME'], subDF['Wind Direction (TAO)'], label = 'TAO', alpha = 0.5)
        ax2.legend()
        
        sns.scatterplot(subDF, x = 'QS_TIME', y = 'Speed Difference (QuikSCAT - TAO)', hue = 'db_speedDirection_label', s = 6, ax = ax3, palette = sns.color_palette("bright"))
        ax3.grid(True)
        ax3.set_title('Speed Difference (QS - TAO) [m/s]')
        
        sns.scatterplot(subDF, x='QS_TIME', y='Direction Difference (QuikSCAT - TAO)', hue = 'db_speedDirection_label', s = 6, ax = ax4, palette = sns.color_palette("bright"))
        ax4.grid(True)
        ax4.set_title('Direction Difference (QS - TAO) [degrees]')
        ax4.set_ylim(-90,90)
        
        ax1.set_title(f'{lon:4d}, {lat:4d}')
        plt.savefig(f'Fig{lon}lon_{lat}lat.png')
        plt.close()

In [ ]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

for task in taskList:
    lat = task[0]
    lon = task[1]
    plotSpeedDirection(lat, lon, lbDF)

In [ ]:
def getGoodBad(lat, lon, df):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    if isPresent:    
        goodCount = len(subDF.loc[subDF['GoodBad'] == 'good'])/len(subDF)*100
        badCount = len(subDF.loc[subDF['GoodBad'] == 'bad'])/len(subDF)*100
        return f' {goodCount:3.0f},{badCount:3.0f}'
    else:
        return ''

def getBadPercent(lat, lon, df):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    
    if isPresent:
        goodCount = len(subDF.loc[subDF['GoodBad'] == 'good'])/len(subDF)*100
        badCount = len(subDF.loc[subDF['GoodBad'] == 'bad'])/len(subDF)*100
        
        return badCount
    else:
        return -999

def getCount(lat, lon, df):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    if isPresent:
        count = len(subDF) #.loc[subDF['GoodBad'] == 'good'])/len(subDF)*100
        return count
    else:
        return -999


def getSpeedCorrelation(lat, lon, df):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    if isPresent:
        QSspeed = subDF['Wind Speed (QuikSCAT)'].to_numpy()
        TAOspeed = subDF['Neutral Wind Speed at 10m (TAO)'].to_numpy()
        corr = np.corrcoef(QSspeed, TAOspeed)[0,1]
        return corr
    else:
        return -999


def getXSpeedCorrelation(lat, lon, df):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    if isPresent:
        QSspeed = subDF['Wind Speed (QuikSCAT)'].to_numpy()
        QSdir = (-subDF['Wind Direction (QuikSCAT)'].to_numpy()-90)%360
        TAOspeed = subDF['Neutral Wind Speed at 10m (TAO)'].to_numpy()
        TAOdir = (-subDF['Wind Direction (TAO)'].to_numpy()-90)%360
        QSspeed_x = QSspeed * np.cos(np.deg2rad(QSdir))
        TAOspeed_x = TAOspeed * np.cos(np.deg2rad(TAOdir))
        corr = np.corrcoef(QSspeed_x, TAOspeed_x)[0,1]
        return corr
    else:
        return -999

def getYSpeedCorrelation(lat, lon, df):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    if isPresent:
        QSspeed = subDF['Wind Speed (QuikSCAT)'].to_numpy()
        QSdir = (-subDF['Wind Direction (QuikSCAT)'].to_numpy()-90)%360
        TAOspeed = subDF['Neutral Wind Speed at 10m (TAO)'].to_numpy()
        TAOdir = (-subDF['Wind Direction (TAO)'].to_numpy()-90)%360
        QSspeed_y = QSspeed * np.sin(np.deg2rad(QSdir))
        TAOspeed_y = TAOspeed * np.sin(np.deg2rad(TAOdir))      
        corr = np.corrcoef(QSspeed_y, TAOspeed_y)[0,1]
        return corr
    else:
        return -999


def getDirCorrelation(lat, lon, df):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    if isPresent:
        QSdir = subDF['Wind Direction (QuikSCAT)'].to_numpy()
        TAOdir = subDF['Wind Direction (TAO)'].to_numpy()
        corr = np.corrcoef(QSdir, TAOdir)[0,1]
        return corr
    else:
        return -999
    

In [ ]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])ss
for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

ntasks = len(taskList)

import cartopy.crs as ccrs
import cartopy.feature as cfeature 
fig, axes = plt.subplots(nrows = 3, 
                       ncols = 1, 
                        figsize = (20,12),
                       subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)}, # "aspect": 1.2}, #1.2},
                       gridspec_kw = {'wspace':0.2, 'hspace':0.5}) 

land = cfeature.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='50m',
    facecolor='lightgrey'  # Set the color to grey
)

ax = axes[0] 

ax.add_feature(land)


ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()
for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, df)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    if txt != '': 
        badPercent = getBadPercent(lat, lon, df)
        #print(badPercent)
        ax.text(xpos, lat-0.5, txt)
        x = ax.scatter(xpos, lat, c=badPercent, vmin=0, vmax=25, s=60, cmap=cm.turbo)
    
cb = plt.colorbar(x, ax = ax)
cb.ax.set_title('% of Bad Matchups')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

ax.text(0.1, 0.95, 'A.  Percentage of Bad Matchups', horizontalalignment='left',
        verticalalignment='center', transform=ax.transAxes,
        weight = 'heavy', fontsize = 15)

###################################################################################################################
ax = axes[1]

ax.add_feature(land)

ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()
for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, df)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    corr = getSpeedCorrelation(lat, lon, df)
    if corr != -999: 
        ax.text(xpos+1, lat-0.5, f'{corr:4.2f}')
        x = ax.scatter(xpos, lat, c=corr, vmin=0.75, vmax=0.95, s=60, cmap=cm.turbo)
    
cb = plt.colorbar(x, ax = ax)
cb.ax.set_title('Corr. Coeff.')

ax.text(0.1, 0.95, 'B.  Wind Speed Correlation', horizontalalignment='left',
        verticalalignment='center', transform=ax.transAxes,
        weight = 'heavy', fontsize = 15)


###################################################################################################################
ax = axes[2]

ax.add_feature(land)

ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()
for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, df)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    corr = getDirCorrelation(lat, lon, df)
    #print(corr)
    if corr != -999: 
        ax.text(xpos+1, lat-0.5, f'{corr:4.2f}')
        x = ax.scatter(xpos, lat, c=corr, vmin=0.40, vmax=0.85, s=60, cmap=cm.turbo)
    
cb = plt.colorbar(x, ax = ax)
cb.ax.set_title('Corr. Coeff.')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

ax.text(0.1, 0.95, 'C.  Wind Direction Correlation', horizontalalignment='left',
        verticalalignment='center', transform=ax.transAxes,
        weight = 'heavy', fontsize = 15)

plt.subplots_adjust(left = 0.01, right = 1, top =0.98 , bottom = 0.02, wspace=0 , hspace=0)
#plt.tight_layout()

#plt.savefig('goodVsBad.png', dpi=100)


In [ ]:
dateTime =df['QS_TIME']
dateTime[0].year

In [ ]:
# cdfDates = num2date(df['time'], 'seconds since 1990-01-01 00:00:00')
# dateTime = np.array([datetime(cdf.year, cdf.month, cdf.day, cdf.hour, cdf.minute) for cdf in cdfDates])
day_of_year = np.array([date.timetuple().tm_yday for date in dateTime])
df['dateTime'] = dateTime
df['day'] = day_of_year

In [ ]:
list(df.keys())

In [ ]:
def getWindSpedStd(lat, lon, df, timeWindow='30min'):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    
    if isPresent:
        mean_wspdStd2hr	 = np.mean(subDF['std. dev. WSPD '+timeWindow])
        return mean_wspdStd2hr
    else:
        return -999


def getWindDirStd(lat, lon, df, timeWindow='30min'):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    
    if isPresent:
        mean_wspdStd2hr	 = np.mean(subDF['std. dev. WDIR '+timeWindow])
        return mean_wspdStd2hr
    else:
        return -999

def getMeanSpeedCorrelation(lat, lon, df, timeWindow='30min'):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    if isPresent:
        QSspeed = subDF['Wind Speed (QuikSCAT)'].to_numpy()
        TAOspeed = subDF['mean WSPD '+timeWindow].to_numpy()
        corr = np.corrcoef(QSspeed, TAOspeed)[0,1]
        return corr
    else:
        return -999


def getMeanDirCorrelation(lat, lon, df, timeWindow='30min'):
    isPresent = False
    lon = (lon + 360)%360
    subDF = df.loc[df['LATITUDE'] == lat]
    subDF = subDF.loc[subDF['LONGITUDE'] == lon]
    if len(subDF) > 0:
        isPresent = True
    if isPresent:
        QSspeed = subDF['Wind Direction (QuikSCAT)'].to_numpy()
        TAOspeed = subDF['mean WDIR '+timeWindow].to_numpy()
        corr = np.corrcoef(QSspeed, TAOspeed)[0,1]
        return corr
    else:
        return -999
    

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
fig, ax = plt.subplots(nrows = 1, 
                       ncols = 1, 
                        figsize = (20,5),
                       subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180), "aspect": 1.2},
                       gridspec_kw = {'wspace':0.2, 'hspace':0.5}) 

land = cfeature.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='50m',
    facecolor='lightgrey'  # Set the color to grey
)

ax.add_feature(land)

#ax = axes[0]
ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()

subDF = df.loc[df['label'] == 0]

for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = str(getCount(lat, lon, subDF))
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    wspdStd = getCount(lat, lon, subDF)
    if wspdStd != -999: 
        #print(wspdStd)
        ax.text(xpos+1, lat-0.5, f'{wspdStd:6.2f}')
        x = ax.scatter(xpos, lat, c=wspdStd, vmin=0, vmax=1000, s=80,  cmap='Reds')
    
cb = plt.colorbar(x)
cb.ax.set_title('Counts of data\n for cluster 0')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

plt.tight_layout()
#plt.savefig('goodVsBad.png', dpi=100)
    


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
fig, ax = plt.subplots(nrows = 1, 
                       ncols = 1, 
                        figsize = (20,5),
                       subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180), "aspect": 1.2},
                       gridspec_kw = {'wspace':0.2, 'hspace':0.5}) 

land = cfeature.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='50m',
    facecolor='lightgrey'  # Set the color to grey
)

ax.add_feature(land)

#ax = axes[0]
ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()

subDF = df.loc[df['label'] == 0]

for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = str(getCount(lat, lon, subDF))
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    wspdStd = getCount(lat, lon, subDF)
    if wspdStd != -999: 
        #print(wspdStd)
        ax.text(xpos+1, lat-0.5, f'{wspdStd:6.2f}')
        x = ax.scatter(xpos, lat, c=wspdStd, vmin=0, vmax=1500, s=80,  cmap='Reds')
    
cb = plt.colorbar(x)
cb.ax.set_title('Counts of data\n for cluster 0')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

plt.tight_layout()
#plt.savefig('goodVsBad.png', dpi=100)
    


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
fig, ax = plt.subplots(nrows = 1, 
                       ncols = 1, 
                        figsize = (20,5),
                       subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180), "aspect": 1.2},
                       gridspec_kw = {'wspace':0.2, 'hspace':0.5}) 

land = cfeature.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='50m',
    facecolor='lightgrey'  # Set the color to grey
)

ax.add_feature(land)

#ax = axes[0]
ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()

subDF = df.loc[df['label'] == 1]

for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = str(getCount(lat, lon, subDF))
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    wspdStd = getCount(lat, lon, subDF)
    if wspdStd != -999: 
        #print(wspdStd)
        ax.text(xpos+1, lat-0.5, f'{wspdStd:6.2f}')
        x = ax.scatter(xpos, lat, c=wspdStd, vmin=0, vmax=1500, s=80,  cmap='Reds')
    
cb = plt.colorbar(x)
cb.ax.set_title('Counts of data\n for cluster 1')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

plt.tight_layout()
#plt.savefig('goodVsBad.png', dpi=100)
    


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
fig, ax = plt.subplots(nrows = 1, 
                       ncols = 1, 
                        figsize = (20,5),
                       subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180), "aspect": 1.2},
                       gridspec_kw = {'wspace':0.2, 'hspace':0.5}) 

land = cfeature.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='50m',
    facecolor='lightgrey'  # Set the color to grey
)

ax.add_feature(land)

#ax = axes[0]
ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()

subDF = df.loc[df['label'].isin([2,3,4,5,6,7])]

for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = str(getCount(lat, lon, subDF))
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    wspdStd = getCount(lat, lon, subDF)
    if wspdStd != -999: 
        #print(wspdStd)
        ax.text(xpos+1, lat-0.5, f'{wspdStd:6.2f}')
        x = ax.scatter(xpos, lat, c=wspdStd, vmin=0, vmax=700, s=80,  cmap='Reds')
    
cb = plt.colorbar(x)
cb.ax.set_title('Counts of data\n for cluster 2,3,4, 5, 6 and 7')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

plt.tight_layout()
#plt.savefig('goodVsBad.png', dpi=100)
    


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
fig, ax = plt.subplots(nrows = 1, 
                       ncols = 1, 
                        figsize = (20,5),
                       subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180), "aspect": 1.2},
                       gridspec_kw = {'wspace':0.2, 'hspace':0.5}) 

land = cfeature.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='50m',
    facecolor='lightgrey'  # Set the color to grey
)

ax.add_feature(land)

#ax = axes[0]
ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()
for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, df)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    wspdStd = getWindSpedStd(lat, lon, df)
    if wspdStd != -999: 
        ax.text(xpos+1, lat-0.5, f'{wspdStd:6.2f}')
        x = ax.scatter(xpos, lat, c=wspdStd, vmin=0.2, vmax=0.4, s=80,  cmap='turbo')
    
cb = plt.colorbar(x)
cb.ax.set_title('wind speed \n std.dev. [m/s]')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

plt.tight_layout()
# plt.savefig('goodVsBad.png', dpi=100)



In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
fig, ax = plt.subplots(nrows = 1, 
                       ncols = 1, 
                        figsize = (20,5),
                       subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180), "aspect": 1.2},
                       gridspec_kw = {'wspace':0.2, 'hspace':0.5}) 

land = cfeature.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='50m',
    facecolor='lightgrey'  # Set the color to grey
)

ax.add_feature(land)

#ax = axes[0]
ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()
for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, df)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    wdirStd = getWindDirStd(lat, lon, df)
    if wdirStd != -999: 
        ax.text(xpos+1, lat-0.5, f'{wdirStd:6.2f}')
        x = ax.scatter(xpos, lat, c=wdirStd, vmin=0, vmax=15, s=80,  cmap='turbo')
    
cb = plt.colorbar(x)
cb.ax.set_title('wind direction \n std.dev. [deg]')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

plt.tight_layout()
# plt.savefig('goodVsBad.png', dpi=100)
    

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
fig, axes = plt.subplots(nrows = 3, 
                       ncols = 1, 
                        figsize = (20,12),
                       subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)}, # "aspect": 1.2}, #1.2},
                       gridspec_kw = {'wspace':0.2, 'hspace':0.5}) 

land = cfeature.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='50m',
    facecolor='lightgrey'  # Set the color to grey
)

ax = axes[0] 

ax.add_feature(land)


ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()
for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, df)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    if txt != '': 
        badPercent = getBadPercent(lat, lon, df)
        #print(badPercent)
        ax.text(xpos, lat-0.5, txt)
        x = ax.scatter(xpos, lat, c=badPercent, vmin=5, vmax=40, s=60, cmap='turbo')
    
cb = plt.colorbar(x, ax = ax)
cb.ax.set_title('% of Bad Matchups')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

ax.text(0.1, 0.95, 'A.  Percentage of Bad Matchups', horizontalalignment='left',
        verticalalignment='center', transform=ax.transAxes,
        weight = 'heavy', fontsize = 15)


ax = axes[1]

ax.add_feature(land)

ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()
for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, df)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    corr = getMeanSpeedCorrelation(lat, lon, df)
    if corr != -999: 
        ax.text(xpos+1, lat-0.5, f'{corr:4.2f}')
        x = ax.scatter(xpos, lat, c=corr, vmin=0.75, vmax=0.92, s=60, cmap='turbo')
    
cb = plt.colorbar(x, ax = ax)
cb.ax.set_title('Corr. Coeff.')

ax.text(0.1, 0.95, 'B.  Wind Speed Correlation', horizontalalignment='left',
        verticalalignment='center', transform=ax.transAxes,
        weight = 'heavy', fontsize = 15)


ax = axes[2]

ax.add_feature(land)

ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()
for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, df)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    corr = getMeanDirCorrelation(lat, lon, df)
    if corr != -999: 
        ax.text(xpos+1, lat-0.5, f'{corr:4.2f}')
        x = ax.scatter(xpos, lat, c=corr, vmin=0.4, vmax=0.86, s=60, cmap='turbo')
    
cb = plt.colorbar(x, ax = ax)
cb.ax.set_title('Corr. Coeff.')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

ax.text(0.1, 0.95, 'C.  Wind Direction Correlation', horizontalalignment='left',
        verticalalignment='center', transform=ax.transAxes,
        weight = 'heavy', fontsize = 15)

plt.subplots_adjust(left = 0.01, right = 1, top =0.98 , bottom = 0.02, wspace=0 , hspace=0)
#plt.tight_layout()

#plt.savefig('goodVsBad.png', dpi=100)
    


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
fig, axes = plt.subplots(nrows = 4, 
                       ncols = 1, 
                        figsize = (20,16),
                       subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)}, # "aspect": 1.2}, #1.2},
                       gridspec_kw = {'wspace':0.2, 'hspace':0.5}) 

land = cfeature.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='50m',
    facecolor='lightgrey'  # Set the color to grey
)

ax = axes[0]

ax.add_feature(land)

ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()

subDF = df.loc[df['GoodBad'] == 'good']

for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, subDF)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    corr = getSpeedCorrelation(lat, lon, subDF)
    if corr != -999: 
        ax.text(xpos+1, lat-0.5, f'{corr:4.2f}', fontsize = 10)
        x1 = ax.scatter(xpos, lat, c=corr, vmin=0.75, vmax=1, s=40, cmap=cm.turbo)
    
cb = plt.colorbar(x1, ax = ax)
cb.ax.set_title('Corr. Coeff.')

ax.text(0.1, 0.95, 'A.  Wind Speed Correlation for Good Matchups', horizontalalignment='left',
        verticalalignment='center', transform=ax.transAxes,
        weight = 'heavy', fontsize = 15)

#########################################################################################################
#########################################################################################################
#########################################################################################################

ax = axes[1]

ax.add_feature(land)

ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()

subDF = df.loc[df['GoodBad'] == 'good']
for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, subDF)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    corr = getDirCorrelation(lat, lon, subDF)
    if corr != -999: 
        ax.text(xpos+1, lat-0.5, f'{corr:4.2f}', fontsize = 10)
        x2 = ax.scatter(xpos, lat, c=corr, vmin=0.75, vmax=1, s=40, cmap=cm.turbo)
    
cb = plt.colorbar(x2, ax = ax)
cb.ax.set_title('Corr. Coeff.')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

ax.text(0.1, 0.95, 'B.  Wind Direction Correlation for Good Matchups', horizontalalignment='left',
        verticalalignment='center', transform=ax.transAxes,
        weight = 'heavy', fontsize = 15)

#########################################################################################################
#########################################################################################################
#########################################################################################################

ax = axes[2]

ax.add_feature(land)

ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()

subDF = df.loc[df['GoodBad'] == 'bad']

for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, subDF)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    corr = getSpeedCorrelation(lat, lon, subDF)
    if corr != -999: 
        ax.text(xpos+1, lat-0.5, f'{corr:4.2f}', fontsize = 10)
        x3 = ax.scatter(xpos, lat, c=corr, vmin=0.75, vmax=1, s=40, cmap=cm.turbo)
    
cb = plt.colorbar(x3, ax = ax)
cb.ax.set_title('Corr. Coeff.')

ax.text(0.1, 0.95, 'C.  Wind Speed Correlation for Bad Matchups', horizontalalignment='left',
        verticalalignment='center', transform=ax.transAxes,
        weight = 'heavy', fontsize = 15)

#########################################################################################################
#########################################################################################################
#########################################################################################################

ax = axes[3]

ax.add_feature(land)

ax.set_extent([140, -70, -5, 5]) 
gridlines = ax.gridlines(draw_labels=True)
ax.coastlines()

subDF = df.loc[df['GoodBad'] == 'bad']
for task in taskList:
    lat = task[0]
    lon = task[1]
    txt = getGoodBad(lat, lon, subDF)
    

    xpos = lon + 180
    
    if xpos > 180:
       xpos -= 360
    corr = getDirCorrelation(lat, lon, subDF)
    if corr != -999: 
        ax.text(xpos+1, lat-0.5, f'{corr:4.2f}', fontsize = 10)
        x4 = ax.scatter(xpos, lat, c=corr, vmin=0.75, vmax=1, s=40, cmap=cm.turbo)
    
cb = plt.colorbar(x4, ax = ax)
cb.ax.set_title('Corr. Coeff.')
#ax.scatter(df['LONGITUDE'], df['LATITUDE'], transform=ccrs.PlateCarree())

ax.text(0.1, 0.95, 'D.  Wind Direction Correlation for Bad Matchups', horizontalalignment='left',
        verticalalignment='center', transform=ax.transAxes,
        weight = 'heavy', fontsize = 15)

plt.subplots_adjust(left = 0.01, right = 1, top =0.98 , bottom = 0.02, wspace=0 , hspace=0)
#plt.tight_layout()

#plt.savefig('goodVsBad_byMatchupLabel.png', dpi=100)

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols =1, sharex = True, figsize=(20,15))

cmap = plt.get_cmap('coolwarm').copy()
cmap.set_bad('white')
nx = 12*7*5 # 168
ny = 10

title = 'For Buoy locations lat > 3 N'
ax = axes[0]
subDF = df.loc[df['LATITUDE'] > 3]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

stack_order = ['good','bad']
sns.histplot(subDF, x='dateTime', bins= nx, hue='GoodBad',  multiple="stack", 
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
sns.move_legend(ax, "upper left")
ax.set_title(title, weight='heavy')


ax = axes[1]
GoodCounts, bin_edges, _ = ax.hist(goodDF['dateTime'], bins=nx, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF['dateTime'], bins=nx, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,50)
ax.set_ylabel('Bad Data %')
################################################################################################

title = 'For Buoy locations lon < -165 E '
ax = axes[2]
subDF = df.loc[df['LONGITUDE'] < (360-165)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x='dateTime', bins= nx, hue='GoodBad',  multiple="stack", 
             hue_order=stack_order, 
             palette = {'good': 'green', 'bad': 'red'}, ax = ax)
sns.move_legend(ax, "upper left")
ax.set_title(title, weight='heavy')

ax = axes[3]
GoodCounts, bin_edges, _ = ax.hist(goodDF['dateTime'], bins=nx, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF['dateTime'], bins=nx, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,50)
ax.set_ylabel('Bad Data %')
################################################################################################

title = 'For Buoy locations lon > -120 E '
ax = axes[4]
subDF = df.loc[df['LONGITUDE'] > (360-120)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x='dateTime', bins= nx, hue='GoodBad', multiple="stack",  
             hue_order=stack_order, 
             palette = {'good': 'green', 'bad': 'red'}, ax = ax)
sns.move_legend(ax, "upper left")
ax.set_title(title, weight='heavy')

ax = axes[5]
GoodCounts, bin_edges, _ = ax.hist(goodDF['dateTime'], bins=nx, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF['dateTime'], bins=nx, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,50)
ax.set_ylabel('Bad Data %')
################################################################################################

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols =1, sharex = True, figsize=(20,15))

cmap = plt.get_cmap('coolwarm').copy()
cmap.set_bad('white')
nx = int(365//3)
ny = 10

title = 'For Buoy locations lat > 3 N'
ax = axes[0]
subDF = df.loc[df['LATITUDE'] > 3]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

stack_order = ['good','bad']
sns.histplot(subDF, x='day', bins= nx, hue='GoodBad',  multiple="stack", 
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
sns.move_legend(ax, "upper left")
ax.set_title(title, weight='heavy')


ax = axes[1]
GoodCounts, bin_edges, _ = ax.hist(goodDF['day'], bins=nx, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF['day'], bins=nx, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,50)
ax.set_ylabel('Bad Data %')
################################################################################################

title = 'For Buoy locations lon < -165 E '
ax = axes[2]
subDF = df.loc[df['LONGITUDE'] < (360-165)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x='day', bins= nx, hue='GoodBad',  multiple="stack", 
             hue_order=stack_order, 
             palette = {'good': 'green', 'bad': 'red'}, ax = ax)
sns.move_legend(ax, "upper left")
ax.set_title(title, weight='heavy')

ax = axes[3]
GoodCounts, bin_edges, _ = ax.hist(goodDF['day'], bins=nx, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF['day'], bins=nx, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,50)
ax.set_ylabel('Bad Data %')
################################################################################################

title = 'For Buoy locations lon > -120 E '
ax = axes[4]
subDF = df.loc[df['LONGITUDE'] > (360-120)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x='day', bins= nx, hue='GoodBad', multiple="stack",  
             hue_order=stack_order, 
             palette = {'good': 'green', 'bad': 'red'}, ax = ax)
sns.move_legend(ax, "upper left")
ax.set_title(title, weight='heavy')

ax = axes[5]
GoodCounts, bin_edges, _ = ax.hist(goodDF['day'], bins=nx, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF['day'], bins=nx, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,50)
ax.set_ylabel('Bad Data %')
ax.set_xlabel('Day of year')
################################################################################################

In [ ]:
subDF

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4,4), sharey=False)

def getPDF(subDF, ax, xlabel, ylabel):
    #ax = axes[0,0]
    cmap = plt.get_cmap('Greys').copy()
    cmap.set_bad('white')
    h1 = ax.hist2d(subDF[xlabel], subDF[ylabel], bins=[50, 50], cmap=cmap, 
                       norm=colors.LogNorm())
    s = sns.kdeplot(subDF, x=xlabel, y=ylabel,
                    levels=20, linewidths=0.5, 
                    palette = sns.color_palette("bright"), ax = ax)
    
    xmin, xmax = subDF[xlabel].min(), subDF[xlabel].max()
    ymin, ymax = subDF[ylabel].min(), subDF[ylabel].max()
    
    xrange = (xmax - xmin)
    xtol = 0.1 * xrange
    
    yrange = (ymax - ymin)
    ytol = 0.1 * yrange
    
    xmin -= xtol
    xmax += xtol
    
    ymin -= ytol
    ymax += ytol
    
    
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    plt.colorbar(h1[3], ax = ax)
    #ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

getPDF(subDF, ax, 'Zonal Neutral Wind Speed at 10m (TAO)', 'Direction Difference (QuikSCAT - TAO)')

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols =4, sharex = False, figsize=(15,12))
histkind = "stack"
cmap = plt.get_cmap('coolwarm').copy()
cmap.set_bad('white')
custom_bins_RH = np.linspace(65,100,50)
custom_bins_SST = np.linspace(20,32,50)
custom_bins_AIRT = np.linspace(15,32,50)
custom_bins_U10x = np.linspace(-15,10,50)
ny = 10

xlabel = 'Relative Humidity (TAO)'
title = 'For Buoy locations lat > 3 N'
ax = axes[0,0]
subDF = df.loc[df['LATITUDE'] > 3]
subDF = subDF.rename(columns = {'GoodBad':'Matchup Agreement'})
# goodDF = subDF.loc[subDF['GoodBad'] == 'good']
# badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF.iloc[::-1], x=xlabel, 
             bins= custom_bins_RH, 
             hue='Matchup Agreement', 
             multiple=histkind,palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)

#ax.set_title(title)
#ax.set_xlabel('TAO Relative Humidity')
ax.grid(visible=True, which='major')

ax = axes[0,1]
xlabel = 'Sea Surface Temperature (TAO)'
custom_bins_SST = np.linspace(25,31,50)

sns.histplot(subDF.iloc[::-1], x=xlabel, 
             bins= custom_bins_SST,
             hue='Matchup Agreement', 
             multiple=histkind,palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#ax.set_xlabel('TAO Sea Surface Temperature')
ax.grid(visible=True, which='major')


ax = axes[0,2]
xlabel = 'Air Temperature (TAO)'
custom_bins_AIRT = np.linspace(23,30,50)

sns.histplot(subDF.iloc[::-1], x=xlabel, 
             bins= custom_bins_AIRT, 
             hue='Matchup Agreement',  
             multiple=histkind,palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#ax.set_xlabel('TAO Air Temperature')
ax.grid(visible=True, which='major')

xlabel = 'Zonal Neutral Wind Speed at 10m (TAO)'
ax = axes[0,3]

sns.histplot(subDF.iloc[::-1], x=xlabel, 
             bins= custom_bins_U10x,
             hue='Matchup Agreement',
             multiple=histkind,palette = {'good': 'green', 'bad': 'red'},
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#ax.set_xlabel('TAO Zonal winds')
ax.grid(visible=True, which='major')

################################################################################################

xlabel = 'Relative Humidity (TAO)'
title = 'For Buoy locations lon < -165 E '
ax = axes[1,0]
subDF = df.loc[df['LONGITUDE'] < (360-165)]
subDF = subDF.rename(columns = {'GoodBad':'Matchup Agreement'})
# goodDF = subDF.loc[subDF['GoodBad'] == 'good']
# badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, 
             bins= custom_bins_RH, 
             hue='Matchup Agreement',  
             multiple=histkind,
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#x.set_xlabel('TAO Relative Humidity')
ax.grid(visible=True, which='major')

ax = axes[1,1]
xlabel = 'Sea Surface Temperature (TAO)'
custom_bins_SST = np.linspace(26,32,50)

sns.histplot(subDF.iloc[::-1], x=xlabel,
             bins= custom_bins_SST, 
             hue='Matchup Agreement',  
             multiple=histkind,
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#ax.set_xlabel('TAO Sea Surface Temperature')
ax.grid(visible=True, which='major')

ax = axes[1,2]
xlabel = 'Air Temperature (TAO)'
custom_bins_AIRT = np.linspace(24,30.5,50)

sns.histplot(subDF.iloc[::-1], x=xlabel, 
             bins= custom_bins_AIRT, 
             hue='Matchup Agreement',  
             multiple=histkind,
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#ax.set_xlabel('TAO Air Temperature')
ax.grid(visible=True, which='major')

xlabel = 'Zonal Neutral Wind Speed at 10m (TAO)'
ax = axes[1,3]

sns.histplot(subDF.iloc[::-1], x=xlabel, 
             bins= custom_bins_U10x, 
             hue='Matchup Agreement',  
             multiple=histkind,
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#ax.set_xlabel('TAO Zonal winds')
ax.grid(visible=True, which='major')


################################################################################################
xlabel = 'Relative Humidity (TAO)'
title = 'For Buoy locations lon > -120 E '

ax = axes[2,0]
subDF = df.loc[df['LONGITUDE'] > (360-120)]
subDF = subDF.rename(columns = {'GoodBad':'Matchup Agreement'})
# goodDF = subDF.loc[subDF['GoodBad'] == 'good']
# badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, 
             bins = custom_bins_RH,  
             hue='Matchup Agreement', 
             multiple=histkind,  
             palette = {'good': 'green', 'bad': 'red'},
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#ax.set_xlabel('TAO Relative Humidity')
ax.grid(visible=True, which='major')
#ax.text(-0.01, 0.33, 'Eastern Pacific', transform=ax.transAxes, rotation=90)

ax = axes[2,1]
xlabel = 'Sea Surface Temperature (TAO)'
custom_bins_SST = np.linspace(20,30,50)

sns.histplot(subDF.iloc[::-1], x=xlabel, 
             bins= custom_bins_SST, 
             hue='Matchup Agreement',  
             multiple=histkind,
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#ax.set_xlabel('TAO Sea Surface Temperature')
ax.grid(visible=True, which='major')

ax = axes[2,2]
xlabel = 'Air Temperature (TAO)'
custom_bins_AIRT = np.linspace(20,29,50)

sns.histplot(subDF.iloc[::-1], x=xlabel, 
             bins= custom_bins_AIRT, 
             hue='Matchup Agreement',  
             multiple=histkind,
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#ax.set_xlabel('TAO Air Temperature')
ax.grid(visible=True, which='major')

xlabel = 'Zonal Neutral Wind Speed at 10m (TAO)'
ax = axes[2,3]

sns.histplot(subDF.iloc[::-1], x=xlabel,
             bins= custom_bins_U10x, 
             hue='Matchup Agreement',  
             multiple=histkind,
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order, 
             ax = ax)
#ax.set_title(title)
#ax.set_xlabel('TAO Zonal winds')
ax.grid(visible=True, which='major')


################################################################################################
plt.subplots_adjust(left=0.05, bottom=0.05, right=0.98, top=0.9, wspace=0.25, hspace=0.25)
#plt.savefig('StackedHistogram.png', dpi = 100)

In [ ]:
xlabel = 'rain_impact'
title = 'For Buoy locations lat > 3 N'
subDF = df.loc[df['LATITUDE'] > 3]
subDF = subDF.rename(columns = {'GoodBad':'Matchup Agreement'})
# goodDF = subDF.loc[subDF['GoodBad'] == 'good']
# badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False)#, 
             #ax = ax)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols =5, sharex = False, figsize=(15,12))
histkind = "stack"
cmap = plt.get_cmap('coolwarm').copy()
cmap.set_bad('white')
custom_bins_RH = np.linspace(65,100,50)
custom_bins_SST = np.linspace(20,32,50)
custom_bins_AIRT = np.linspace(15,32,50)
custom_bins_U10x = np.linspace(-15,10,50)
ny = 10

xlabel = 'Relative Humidity (TAO)'
title = 'For Buoy locations lat > 3 N'
ax = axes[0,0]
subDF = df.loc[df['LATITUDE'] > 3]
subDF = subDF.rename(columns = {'GoodBad':'Matchup Agreement'})
# goodDF = subDF.loc[subDF['GoodBad'] == 'good']
# badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)

#ax.set_title(title)
#ax.set_xlabel('TAO Relative Humidity')
ax.grid(visible=True, which='major')

ax = axes[0,1]
xlabel = 'Sea Surface Temperature (TAO)'
custom_bins_SST = np.linspace(25,31,50)

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Sea Surface Temperature')
ax.grid(visible=True, which='major')


ax = axes[0,2]
xlabel = 'Air Temperature (TAO)'
custom_bins_AIRT = np.linspace(23,30,50)

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)

#ax.set_title(title)
#ax.set_xlabel('TAO Air Temperature')
ax.grid(visible=True, which='major')

xlabel = 'Zonal Neutral Wind Speed at 10m (TAO)'
ax = axes[0,3]

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Zonal winds')
ax.grid(visible=True, which='major')

xlabel = 'rain_impact'
ax = axes[0,4]

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Zonal winds')
ax.grid(visible=True, which='major')

################################################################################################

xlabel = 'Relative Humidity (TAO)'
title = 'For Buoy locations lon < -165 E '
ax = axes[1,0]
subDF = df.loc[df['LONGITUDE'] < (360-165)]
subDF = subDF.rename(columns = {'GoodBad':'Matchup Agreement'})
# goodDF = subDF.loc[subDF['GoodBad'] == 'good']
# badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#x.set_xlabel('TAO Relative Humidity')
ax.grid(visible=True, which='major')

ax = axes[1,1]
xlabel = 'Sea Surface Temperature (TAO)'
custom_bins_SST = np.linspace(26,32,50)

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Sea Surface Temperature')
ax.grid(visible=True, which='major')

ax = axes[1,2]
xlabel = 'Air Temperature (TAO)'
custom_bins_AIRT = np.linspace(24,30.5,50)

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Air Temperature')
ax.grid(visible=True, which='major')

xlabel = 'Zonal Neutral Wind Speed at 10m (TAO)'
ax = axes[1,3]

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Zonal winds')
ax.grid(visible=True, which='major')

xlabel = 'rain_impact'
ax = axes[1,4]

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Zonal winds')
ax.grid(visible=True, which='major')


################################################################################################
xlabel = 'Relative Humidity (TAO)'
title = 'For Buoy locations lon > -120 E '

ax = axes[2,0]
subDF = df.loc[df['LONGITUDE'] > (360-120)]
subDF = subDF.rename(columns = {'GoodBad':'Matchup Agreement'})
# goodDF = subDF.loc[subDF['GoodBad'] == 'good']
# badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Relative Humidity')
ax.grid(visible=True, which='major')
#ax.text(-0.01, 0.33, 'Eastern Pacific', transform=ax.transAxes, rotation=90)

ax = axes[2,1]
xlabel = 'Sea Surface Temperature (TAO)'
custom_bins_SST = np.linspace(20,30,50)

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Sea Surface Temperature')
ax.grid(visible=True, which='major')

ax = axes[2,2]
xlabel = 'Air Temperature (TAO)'
custom_bins_AIRT = np.linspace(20,29,50)

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Air Temperature')
ax.grid(visible=True, which='major')

xlabel = 'Zonal Neutral Wind Speed at 10m (TAO)'
ax = axes[2,3]

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Zonal winds')
ax.grid(visible=True, which='major')

xlabel = 'rain_impact'
ax = axes[2,4]

sns.kdeplot(subDF.iloc[::-1], x=xlabel, 
             #bins= custom_bins_RH, 
             hue='Matchup Agreement',
             #kind='kde',
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,#)
             common_norm=False, 
             ax = ax)
            
#ax.set_title(title)
#ax.set_xlabel('TAO Zonal winds')
ax.grid(visible=True, which='major')


################################################################################################
plt.subplots_adjust(left=0.05, bottom=0.05, right=0.98, top=0.9, wspace=0.25, hspace=0.25)
#plt.savefig('StackedHistogram.png', dpi = 100)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols =4, sharex = True, figsize=(20,10))

cmap = plt.get_cmap('coolwarm').copy()
cmap.set_bad('white')
custom_bins = np.linspace(65,100,50)
ny = 10

xlabel = 'Relative Humidity (TAO)'
title = 'For Buoy locations lat > 3 N'
ax = axes[0,0]
subDF = df.loc[df['LATITUDE'] > 3]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF.iloc[::-1], x=xlabel, bins= custom_bins, hue='GoodBad',  multiple="stack", 
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,
             ax = ax)
ax.set_title(title)
ax.grid(visible=True, which='major')

ax = axes[0,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')

ax = axes[0,2]
getPDF(subDF, ax, xlabel, 'Direction Difference (QuikSCAT - TAO)')
ax.set_ylabel('direction Diff.')

ax = axes[0,3]
getPDF(subDF, ax, xlabel, 'Speed Difference (QuikSCAT - TAO)')
ax.set_ylabel('speed Diff.')
################################################################################################


title = 'For Buoy locations lon < -165 E '
ax = axes[1,0]
subDF = df.loc[df['LONGITUDE'] < (360-165)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, bins= custom_bins, hue='GoodBad',  multiple="stack", 
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,
             ax = ax)
ax.set_title(title)
ax.grid(visible=True, which='major')

title = 'For Buoy locations lon > -120 E '

ax = axes[1,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')

ax = axes[1,2]
getPDF(subDF, ax, xlabel, 'Direction Difference (QuikSCAT - TAO)')
ax.set_ylabel('direction Diff.')

ax = axes[1,3]
getPDF(subDF, ax, xlabel, 'Speed Difference (QuikSCAT - TAO)')
ax.set_ylabel('speed Diff.')
################################################################################################


ax = axes[2,0]
subDF = df.loc[df['LONGITUDE'] > (360-120)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, bins = custom_bins,  hue='GoodBad', multiple="stack",  
             palette = {'good': 'green', 'bad': 'red'},
             hue_order=stack_order,
             ax = ax)
ax.set_title(title)
#ax.set_xlabel('TAO zonal Wind')
ax.grid(visible=True, which='major')

ax = axes[2,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')
ax.set_xlabel(xlabel)

ax = axes[2,2]
getPDF(subDF, ax, xlabel, 'Direction Difference (QuikSCAT - TAO)')
ax.set_ylabel('direction Diff.')

ax = axes[2,3]
getPDF(subDF, ax, xlabel, 'Speed Difference (QuikSCAT - TAO)')
ax.set_ylabel('speed Diff.')
################################################################################################

plt.subplots_adjust(left=0.05, bottom=0.05, right=0.98, top=0.9, wspace=0.20, hspace=0.15)
#plt.savefig('QuikSCAT_RH.png', dpi = 100)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols =4, sharex = False, figsize=(20,10))

cmap = plt.get_cmap('coolwarm').copy()
cmap.set_bad('white')
custom_bins = np.linspace(25,32,50)
ny = 10

xlabel = 'Sea Surface Temperature (TAO)'
title = 'For Buoy locations lat > 3 N'
ax = axes[0,0]
subDF = df.loc[df['LATITUDE'] > 3]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF.iloc[::-1], x=xlabel, bins= custom_bins, hue='GoodBad',  multiple="stack", 
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,
             ax = ax)
ax.set_title(title)
ax.grid(visible=True, which='major')

ax = axes[0,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')

ax = axes[0,2]
getPDF(subDF, ax, xlabel, 'Direction Difference (QuikSCAT - TAO)')
ax.set_ylabel('direction Diff.')

ax = axes[0,3]
getPDF(subDF, ax, xlabel, 'Speed Difference (QuikSCAT - TAO)')
ax.set_ylabel('speed Diff.')
################################################################################################

custom_bins = np.linspace(25,31,50)
title = 'For Buoy locations lon < -165 E '
ax = axes[1,0]
subDF = df.loc[df['LONGITUDE'] < (360-165)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, bins= custom_bins, hue='GoodBad',  multiple="stack", 
             palette = {'good': 'green', 'bad': 'red'},
             hue_order=stack_order,
             ax = ax)
ax.set_title(title)
ax.grid(visible=True, which='major')

title = 'For Buoy locations lon > -120 E '

ax = axes[1,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')

ax = axes[1,2]
getPDF(subDF, ax, xlabel, 'Direction Difference (QuikSCAT - TAO)')
ax.set_ylabel('direction Diff.')

ax = axes[1,3]
getPDF(subDF, ax, xlabel, 'Speed Difference (QuikSCAT - TAO)')
ax.set_ylabel('speed Diff.')
################################################################################################

custom_bins = np.linspace(20,32,50)
ax = axes[2,0]
subDF = df.loc[df['LONGITUDE'] > (360-120)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, bins = custom_bins,  hue='GoodBad', multiple="stack",  
             palette = {'good': 'green', 'bad': 'red'},
             hue_order=stack_order,
             ax = ax)
ax.set_title(title)
#ax.set_xlabel('TAO zonal Wind')
ax.grid(visible=True, which='major')

ax = axes[2,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')
ax.set_xlabel(xlabel)

ax = axes[2,2]
getPDF(subDF, ax, xlabel, 'Direction Difference (QuikSCAT - TAO)')
ax.set_ylabel('direction Diff.')

ax = axes[2,3]
getPDF(subDF, ax, xlabel, 'Speed Difference (QuikSCAT - TAO)')
ax.set_ylabel('speed Diff.')
################################################################################################

plt.subplots_adjust(left=0.05, bottom=0.05, right=0.98, top=0.9, wspace=0.20, hspace=0.15)
#plt.savefig('QuikSCAT_SST.png', dpi = 100)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols =2, sharex = False, figsize=(10,10))

cmap = plt.get_cmap('coolwarm').copy()
cmap.set_bad('white')
custom_bins = np.linspace(25,32,50)
ny = 10

xlabel = 'Sea Surface Temperature (TAO)'
title = 'For Buoy locations lat > 3 N'
ax = axes[0,0]
subDF = df.loc[df['LATITUDE'] > 3]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF.iloc[::-1], x=xlabel, bins= custom_bins, hue='GoodBad',  multiple="stack", 
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,
             ax = ax)
ax.set_xlabel('')
ax.set_title(title)
ax.grid(visible=True, which='major')

ax = axes[0,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')
################################################################################################

custom_bins = np.linspace(25,31,50)
title = 'For Buoy locations lon < -165 E '
ax = axes[1,0]
subDF = df.loc[df['LONGITUDE'] < (360-165)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, bins= custom_bins, hue='GoodBad',  multiple="stack", 
             palette = {'good': 'green', 'bad': 'red'}, 
             hue_order=stack_order,
             ax = ax)
ax.set_xlabel('')
ax.set_title(title)
ax.grid(visible=True, which='major')

title = 'For Buoy locations lon > -120 E '

ax = axes[1,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')
################################################################################################

custom_bins = np.linspace(20,32,50)
ax = axes[2,0]
subDF = df.loc[df['LONGITUDE'] > (360-120)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, bins = custom_bins,  hue='GoodBad', multiple="stack",  
             palette = {'good': 'green', 'bad': 'red'},
             hue_order=stack_order,
             ax = ax)
ax.set_title(title)
#ax.set_xlabel('TAO zonal Wind')
ax.grid(visible=True, which='major')
ax.set_xlabel(r'$\degree C$')

ax = axes[2,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')
ax.set_xlabel(r'$\degree C$')
################################################################################################


In [ ]:
fig, axes = plt.subplots(nrows=3, ncols =4, sharex = True, figsize=(20,10))

cmap = plt.get_cmap('coolwarm').copy()
cmap.set_bad('white')
custom_bins = np.linspace(-15,10,50)
ny = 10

xlabel = 'Zonal Neutral Wind Speed at 10m (TAO)'
title = 'For Buoy locations lat > 3 N'
ax = axes[0,0]
subDF = df.loc[df['LATITUDE'] > 3]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF.iloc[::-1], x=xlabel, bins= custom_bins, hue='GoodBad',  multiple="stack", 
             palette = {'good': 'green', 'bad': 'red'},
             hue_order=stack_order,
              ax = ax)
ax.set_title(title)
ax.grid(visible=True, which='major')

ax = axes[0,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')

ax = axes[0,2]
getPDF(subDF, ax, xlabel, 'Direction Difference (QuikSCAT - TAO)')
ax.set_ylabel('direction Diff.')

ax = axes[0,3]
getPDF(subDF, ax, xlabel, 'Speed Difference (QuikSCAT - TAO)')
ax.set_ylabel('speed Diff.')
################################################################################################

#custom_bins = np.linspace(25,31,50)
title = 'For Buoy locations lon < -165 E '
ax = axes[1,0]
subDF = df.loc[df['LONGITUDE'] < (360-165)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, bins= custom_bins, hue='GoodBad',  multiple="stack", 
             palette = {'good': 'green', 'bad': 'red'},
             hue_order=stack_order,
             ax = ax)
ax.set_title(title)
ax.grid(visible=True, which='major')

title = 'For Buoy locations lon > -120 E '

ax = axes[1,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')

ax = axes[1,2]
getPDF(subDF, ax, xlabel, 'Direction Difference (QuikSCAT - TAO)')
ax.set_ylabel('direction Diff.')

ax = axes[1,3]
getPDF(subDF, ax, xlabel, 'Speed Difference (QuikSCAT - TAO)')
ax.set_ylabel('speed Diff.')
################################################################################################

#custom_bins = np.linspace(20,32,50)
ax = axes[2,0]
subDF = df.loc[df['LONGITUDE'] > (360-120)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, bins = custom_bins,  hue='GoodBad', multiple="stack",  
             palette = {'good': 'green', 'bad': 'red'},
             hue_order=stack_order,
             ax = ax)
ax.set_title(title)
#ax.set_xlabel('TAO zonal Wind')
ax.grid(visible=True, which='major')
ax.set_xlabel('Zonal Wind [m/s]')

ax = axes[2,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')
ax.set_xlabel('Zonal Wind [m/s]')

ax = axes[2,2]
getPDF(subDF, ax, xlabel, 'Direction Difference (QuikSCAT - TAO)')
ax.set_ylabel('direction Diff.')
ax.set_xlabel('Zonal Wind [m/s]')

ax = axes[2,3]
getPDF(subDF, ax, xlabel, 'Speed Difference (QuikSCAT - TAO)')
ax.set_xlabel('Zonal Wind [m/s]')
ax.set_ylabel('speed Diff.')
################################################################################################

plt.subplots_adjust(left=0.05, bottom=0.05, right=0.98, top=0.9, wspace=0.20, hspace=0.15)
#plt.savefig('QuikSCAT_zonalWinds.png', dpi = 100)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols =2, sharex = True, figsize=(10,10))

cmap = plt.get_cmap('coolwarm').copy()
cmap.set_bad('white')
custom_bins = np.linspace(-15,10,50)
ny = 10

xlabel = 'Zonal Neutral Wind Speed at 10m (TAO)'
title = 'For Buoy locations lat > 3 N'
ax = axes[0,0]
subDF = df.loc[df['LATITUDE'] > 3]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF.iloc[::-1], x=xlabel, bins= custom_bins, hue='GoodBad',  multiple="stack", 
             #palette = {'good': 'green', 'bad': 'red'}, ax = ax)
ax.set_xlabel('')
ax.set_title(title)
ax.grid(visible=True, which='major')

ax = axes[0,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')
################################################################################################

#custom_bins = np.linspace(-15,10,50)
title = 'For Buoy locations lon < -165 E '
ax = axes[1,0]
subDF = df.loc[df['LONGITUDE'] < (360-165)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, bins= custom_bins, hue='GoodBad',  multiple="stack", 
             #palette = {'good': 'green', 'bad': 'red'}, ax = ax)
ax.set_xlabel('')
ax.set_title(title)
ax.grid(visible=True, which='major')

title = 'For Buoy locations lon > -120 E '

ax = axes[1,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')
################################################################################################

#custom_bins = np.linspace(-15,15,50)
ax = axes[2,0]
subDF = df.loc[df['LONGITUDE'] > (360-120)]
goodDF = subDF.loc[subDF['GoodBad'] == 'good']
badDF = subDF.loc[subDF['GoodBad'] == 'bad']

sns.histplot(subDF, x=xlabel, bins = custom_bins,  hue='GoodBad', multiple="stack",  
             #palette = {'good': 'green', 'bad': 'red'}, ax = ax)
ax.set_title(title)
#ax.set_xlabel('TAO zonal Wind')
ax.grid(visible=True, which='major')
ax.set_xlabel('Zonal Wind [m/s]')

ax = axes[2,1]
GoodCounts, bin_edges, _ = ax.hist(goodDF[xlabel], bins=custom_bins, color='blue', alpha=0)  # Use 
BadCounts, bin_edges, _ = ax.hist(badDF[xlabel], bins=custom_bins, color='Red', alpha=0)  # Use 
BadPercent = BadCounts/(GoodCounts+BadCounts) * 100
ax.plot(bin_edges[0:-1], BadPercent)
ax.set_ylim(0,100)
ax.set_ylabel('Bad Data %')
ax.set_xlabel('Zonal Wind [m/s]')
################################################################################################


In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4,4), sharey=False)

def getPDF(subDF, ax, xlabel, ylabel):
    #ax = axes[0,0]
    cmap = plt.get_cmap('Greys').copy()
    cmap.set_bad('white')
    h1 = ax.hist2d(subDF[xlabel], subDF[ylabel], bins=[50, 50], cmap=cmap, 
                       norm=colors.LogNorm())
    s = sns.kdeplot(subDF, x=xlabel, y=ylabel,
                    levels=20, linewidths=0.5, 
                    palette = sns.color_palette("bright"), ax = ax)
    
    xmin, xmax = subDF[xlabel].min(), subDF[xlabel].max()
    ymin, ymax = subDF[ylabel].min(), subDF[ylabel].max()
    
    xrange = (xmax - xmin)
    xtol = 0.1 * xrange
    
    yrange = (ymax - ymin)
    ytol = 0.1 * yrange
    
    xmin -= xtol
    xmax += xtol
    
    ymin -= ytol
    ymax += ytol
    
    
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    plt.colorbar(h1[3], ax = ax)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

getPDF(subDF, ax, 'Zonal Neutral Wind Speed at 10m (TAO)', 'Direction Difference (QuikSCAT - TAO)')

In [ ]:
xlabel = 'Speed Difference (QuikSCAT - TAO)'
ylabel = 'Direction Difference (QuikSCAT - TAO)'
nx = 100
ny = 100
fig, ax = plt.subplots(figsize=(10,10))
cmap = plt.get_cmap('Greys').copy()
cmap.set_bad('white')

subDF = df.loc[df['label'].isin([0,2,3,5])]#,6,8])]

h1 = ax.hist2d(subDF[xlabel], subDF[ylabel], bins=[nx, ny], cmap=cmap, 
                   norm=colors.LogNorm())
s = sns.kdeplot(subDF, x=xlabel, y=ylabel, common_norm=False,
                hue='label',levels=20, linewidths=0.5, 
                palette = sns.color_palette("bright"), ax = ax)

xmin, xmax = subDF[xlabel].min(), subDF[xlabel].max()
ymin, ymax = subDF[ylabel].min(), subDF[ylabel].max()

xrange = (xmax - xmin)
xtol = 0.1 * xrange

yrange = (ymax - ymin)
ytol = 0.1 * yrange

xmin -= xtol
xmax += xtol

ymin -= ytol
ymax += ytol


ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
plt.colorbar(h1[3], ax = ax)

In [ ]:
# xlabel = 'Speed Difference (QuikSCAT - TAO)'
# ylabel = 'Direction Difference (QuikSCAT - TAO)'
# s = sns.kdeplot(df, x=xlabel, y = ylabel,levels=10, 
#                 hue='label', #common_norm=False,
#                 palette = sns.color_palette("bright"))

In [ ]:
def pdfPlotTest(xlabel, ylabel, subDF, nx = 50, ny =50):
    fname = f'{xlabel}_{ylabel}_density.png'
    
    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(10,3), sharey=True)
    
    ax = axes[0]
    s = sns.kdeplot(subDF, x=xlabel, y=ylabel, 
                    hue='label',levels=20, thres=0.1, linewidths=0.5, 
                    palette = sns.color_palette("bright"), ax = ax)

    ssubDF = subDF.loc[subDF['label'] == 0]
    ax = axes[1]
    h1 = ax.hist2d(ssubDF[xlabel], ssubDF[ylabel], bins=[nx, ny], cmap=plt.get_cmap('Reds'))
    s = sns.kdeplot(ssubDF, x=xlabel, y=ylabel, 
                    levels=5, thres=0.1, linewidths=0.5, 
                    color = sns.color_palette("bright")[0], ax = ax)
    ax.set_xlabel(xlabel)
    ax.set_xlim(axes[0].get_xlim())
    plt.colorbar(h1[3], ax = ax)
    

    ssubDF = subDF.loc[subDF['label'] == 1]
    ax = axes[2]
    h2 = ax.hist2d(ssubDF[xlabel], ssubDF[ylabel], bins=[nx, ny], cmap=plt.get_cmap('Reds'))
    s = sns.kdeplot(ssubDF, x=xlabel, y=ylabel, 
                    levels=5, thres=0.1, linewidths=0.5, 
                    color = sns.color_palette("bright")[1], ax = ax)
    ax.set_xlabel(xlabel)
    ax.set_xlim(axes[0].get_xlim())
    plt.colorbar(h2[3], ax = ax)

    ssubDF = subDF.loc[subDF['label'] == 2]
    ax = axes[3]
    h3 = ax.hist2d(ssubDF[xlabel], ssubDF[ylabel], bins=[nx, ny], cmap=plt.get_cmap('Reds'))
    s = sns.kdeplot(ssubDF, x=xlabel, y=ylabel, 
                    levels=5, thres=0.1, linewidths=0.5, 
                    color = sns.color_palette("bright")[2], ax = ax)
    ax.set_xlabel(xlabel)
    ax.set_xlim(axes[0].get_xlim())
    plt.colorbar(h3[3], ax = ax)

    plt.subplots_adjust(left = 0.1, right=0.95, top = 0.95, bottom=0.15, wspace=0.2)
    fig.savefig(fname, dpi = 100)
    #plt.close()

In [ ]:
def pdfPlotTest(xlabel, ylabel, subDF, nx = 50, ny =50):
    fname = f'{xlabel}_{ylabel}_density_errorsOnly_of_each_cluster.png'
    
    fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20,8), sharey=False)
    
    ax = axes[0,0]
    cmap = plt.get_cmap('Greys').copy()
    cmap.set_bad('white')
    h1 = ax.hist2d(subDF[xlabel], subDF[ylabel], bins=[nx, ny], cmap=cmap, 
                       norm=colors.LogNorm())
    s = sns.kdeplot(subDF, x=xlabel, y=ylabel, common_norm=False,
                    hue='label',levels=20, linewidths=0.5, 
                    palette = sns.color_palette("bright"), ax = ax)

    xmin, xmax = subDF[xlabel].min(), subDF[xlabel].max()
    ymin, ymax = subDF[ylabel].min(), subDF[ylabel].max()
    
    xrange = (xmax - xmin)
    xtol = 0.1 * xrange

    yrange = (ymax - ymin)
    ytol = 0.1 * yrange

    xmin -= xtol
    xmax += xtol

    ymin -= ytol
    ymax += ytol


    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    plt.colorbar(h1[3], ax = ax)
    ax.set_xlabel(None)
    
    
    for i in range(1,10):
        ii = int(i//5)
        jj = int(i%5)
        ssubDF = subDF.loc[subDF['label'] == i-1]
        ax = axes[ii, jj]
        h1 = ax.hist2d(ssubDF[xlabel], ssubDF[ylabel], bins=[nx, ny], cmap=plt.get_cmap('Greys'), 
                       norm=colors.LogNorm())
        s = sns.kdeplot(ssubDF, x=xlabel, y=ylabel, 
                        levels=10, linewidths=0.5, 
                        color = 'Red', ax = ax)
        if ii == 1:
            ax.set_xlabel(xlabel)
        else:
            ax.set_xlabel(None)

        if jj == 0:
            ax.set_ylabel(ylabel)
        else:
            ax.set_ylabel(None)
            
        xmin, xmax = ssubDF[xlabel].min(), ssubDF[xlabel].max()
        ymin, ymax = ssubDF[ylabel].min(), ssubDF[ylabel].max()
        
        xrange = (xmax - xmin)
        xtol = 0.1 * xrange
    
        yrange = (ymax - ymin)
        ytol = 0.1 * yrange
    
        xmin -= xtol
        xmax += xtol
    
        ymin -= ytol
        ymax += ytol
        
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_title(f'label = {i-1:d}')
        plt.colorbar(h1[3], ax = ax)
    

    plt.subplots_adjust(left = 0.1, right=0.95, top = 0.95, bottom=0.15, wspace=0.2)
    fig.savefig(fname, dpi = 100)
    #plt.close()

In [ ]:
pdfPlotTest('Speed Difference (QuikSCAT - TAO)', 'Direction Difference (QuikSCAT - TAO)', df, nx = 50, ny =50)

In [ ]:
sns.kdeplot(subDF, x='Speed Difference (QuikSCAT - TAO)', y='Direction Difference (QuikSCAT - TAO)', 
                    hue='label',levels=20, linewidths=0.5, 
                    palette = sns.color_palette("bright"))

In [ ]:
for i in range(len(subDFheaders)-1):
    for j in range(i+1, len(subDFheaders)):
        xlabel = subDFheaders[i]
        ylabel = subDFheaders[j]:
        if xlabel != ylabel:
            print(xlabel, ylabel)
            pdfPlotTest(xlabel, ylabel, subDF)

In [ ]:
testDF = df.loc[df['label'] == 1]
testDF.head()

In [ ]:
def pdfPlot1d(subDF, var):
    fname = f'{var}_density.png'
    fig, ax = plt.subplots(figsize=(10,6))
    s = sns.kdeplot(subDF, x=var, hue='label', ax=ax)
    fig.savefig(fname, dpi = 100)
    plt.close()

In [ ]:
for xlabel in subDFheaders:
    print(xlabel)
    pdfPlot1d(subDF, xlabel)

In [ ]:
sns.histplot(df, x='Speed Difference (QuikSCAT - TAO)', y='Direction Difference (QuikSCAT - TAO)')#, hue='label')

In [ ]:
sns.kdeplot(df, x='absSpeedDiff', y='absDirectionDiff', hue='label', levels=20)

In [ ]:
fig, axes = plt.subplots(nrows=1 , ncols=2, figsize=(20,5))

ax = axes[0]
sns.histplot(df[df['label'].isin([0,1,8])], x='Neutral Wind Speed at 10m (TAO)', y='Wind Speed (QuikSCAT)', cbar=True, ax= ax)

ax = axes[1]
sns.histplot(df[df['label'].isin([0,1,8])], x='Wind Direction (TAO)', y='Wind Direction (QuikSCAT)', cbar=True, ax= ax)


In [ ]:
subset = allDS.isel(count = np.arange(100))

In [ ]:
plt.figure()
for count in subset['count']:
    subset['QS-Buoy'].isel(count = count) 

In [ ]:
diff = subset['retrieved_wind_direction'] - subset['WDIR']
diff2 = subset['retrieved_wind_speed'] - subset['WSPD']

In [ ]:
for i in range(10,20):
    diff.isel(count=i).plot(ylim = (-20,20))
    

In [ ]:
for i in range(10,20):
    diff2.isel(count=i).plot(ylim = (-2,2))